This is going to be so amazing you guys

In [1]:
import requests
import pdf2image
import pytesseract
import nltk
import hashlib
from neo4j import GraphDatabase
import pandas as pd
from transformers import AutoTokenizer
from zero_shot_re import RelTaggerModel, RelationExtractor
import itertools

C:\Users\Briefgarde\anaconda3\envs\semesterproject\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertModel: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Briefgarde\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
pdf = requests.get('https://arxiv.org/pdf/2312.09317.pdf') #A parallelized cellular Potts model that enables simulations at
# tissue scale
doc = pdf2image.convert_from_bytes(pdf.content)
# Get the article text
article = []
for page_number, page_data in enumerate(doc):
    txt = pytesseract.image_to_string(page_data).encode("utf-8")
    # Sixth page are only references
    if page_number < 21:
        article.append(txt.decode("utf-8"))
article_txt = " ".join(article)
article_txt

"arX1v:2312.09317v1 [q-bio.TO] 14 Dec 2023\n\nA parallelized cellular Potts model that enables simulations at\ntissue scale\n\nShabaz Sultan!?, Sapna Devi?, Scott N. Mueller?, Johannes Textor!?\n\n! Data Science group, Institute for Computing and Information Sciences, Radboud University, Nijmegen,\nThe Netherlands\n\n? Department of Medical BioSciences, Radboud University Medical Center, Nijmegen, The Netherlands\n\n3 Department of Microbiology and Immunology, The University of Melbourne, at the Peter Doherty\nInstitute for Infection and Immunity, Melbourne, Victoria, Australia\n\nSummary\n\nThe Cellular Potts Model (CPM) is a widely used simulation paradigm for systems of interacting cells that\nhas been used to study scenarios ranging from plant development to morphogenesis, tumor growth and\ncell migration. Despite their wide use, CPM simulations are considered too computationally intensive\nfor three-dimensional (3D) models at organ scale. CPMs have been difficult to parallelise be

In [4]:
def clean_text(text):
    """Remove section titles and figure descriptions from text"""
    clean = "\n".join([row for row in text.split("\n") if
                      (len(row.split(" "))) > 3 and not (row.startswith("(a)"))
                      and not row.startswith("Figure")])
    return clean

text = article_txt.split("Introduction")[1]
ctext = clean_text(text)
sentences = nltk.tokenize.sent_tokenize(ctext)
sentences

['The CPM!',
 'is a powerful simulation method that is known for its ability to simulate a wide range\nof biological scenarios.',
 'When modelling tissue environments, many researchers wish to consider both\nhe complex morphology of single cells, and the behavior of these cells as part of larger collectives.',
 'The appeal of the CPM lies partly in its ability to integrate both these aspects in a single model.',
 'As such, it has been applied to diverse problems including tumor growth,” vascularization,® wound\nhealing,* and morphogenesis.>*® Many extensions and hybrid model variants are available in the literature\n‘0, for example, add support for cell motility,” cell signalling,!° and finite-element based mechanical\ninteractions.!!',
 'Thus, the CPM can be described as a generic platform upon which a wide range of models\ncan be built, possibly explaining its popularity in the tissue modelling field.',
 'For computational reasons, CPM models are often built using a small number of c

In [8]:
def query_plain(text, url="http://bern2.korea.ac.kr/plain"):
 """Biomedical entity linking API"""
 return requests.post(url, json={'text':str(text)}).json()


def query_plain2(text, url="http://bern2.korea.ac.kr/plain"):
    """Biomedical entity linking API"""
    try:
        response = requests.post(url, json={'text': str(text)})
        response.raise_for_status()  # Raise an error for bad responses (4xx and 5xx)
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
        return None

In [9]:
%%time

entity_list = []
print(len(sentences))

i=0
# The last sentence is invalid
for s in sentences[:-1]:
    print("doing sentence : " + str(i))
    entity_list.append(query_plain2(s))
    i += 1

print("all calls done")
print(entity_list)




376
doing sentence : 0
doing sentence : 1
doing sentence : 2
doing sentence : 3
doing sentence : 4
doing sentence : 5
doing sentence : 6
doing sentence : 7
doing sentence : 8
doing sentence : 9
doing sentence : 10
doing sentence : 11
doing sentence : 12
doing sentence : 13
doing sentence : 14
doing sentence : 15
doing sentence : 16
doing sentence : 17
doing sentence : 18
doing sentence : 19
doing sentence : 20
doing sentence : 21
doing sentence : 22
doing sentence : 23
doing sentence : 24
doing sentence : 25
doing sentence : 26
doing sentence : 27
doing sentence : 28
doing sentence : 29
doing sentence : 30
doing sentence : 31
doing sentence : 32
doing sentence : 33
doing sentence : 34
doing sentence : 35
doing sentence : 36
doing sentence : 37
doing sentence : 38
doing sentence : 39
doing sentence : 40
doing sentence : 41
doing sentence : 42
doing sentence : 43
doing sentence : 44
doing sentence : 45
doing sentence : 46
doing sentence : 47
doing sentence : 48
doing sentence : 49
doing 

doing sentence : 374
all calls done
[{'annotations': [], 'text': 'The CPM!', 'timestamp': 'Fri Dec 22 09:30:40 +0000 2023'}, {'annotations': [], 'text': 'is a powerful simulation method that is known for its ability to simulate a wide range of biological scenarios.', 'timestamp': 'Fri Dec 22 09:30:41 +0000 2023'}, {'annotations': [{'id': ['CUI-less'], 'is_neural_normalized': False, 'mention': 'single cells', 'obj': 'cell_type', 'prob': 0.7551743984222412, 'span': {'begin': 100, 'end': 112}}], 'text': 'When modelling tissue environments, many researchers wish to consider both he complex morphology of single cells, and the behavior of these cells as part of larger collectives.', 'timestamp': 'Fri Dec 22 09:30:43 +0000 2023'}, {'annotations': [], 'text': 'The appeal of the CPM lies partly in its ability to integrate both these aspects in a single model.', 'timestamp': 'Fri Dec 22 09:30:44 +0000 2023'}, {'annotations': [{'id': ['mesh:D009369'], 'is_neural_normalized': False, 'mention': 'tu

In [11]:
parsed_entities = []

filtered_entity_list = [entities for entities in entity_list if entities is not None]

for entities in filtered_entity_list:
    e = []
    
    # If there are no entities in the text
    if not entities.get('annotations'):
        parsed_entities.append({
            'text': entities['text'],
            'text_sha256': hashlib.sha256(entities['text'].encode('utf-8')).hexdigest()
        })
        continue
    
    for entity in entities['annotations']:
        other_ids = [id for id in entity['id'] if not id.startswith("BERN")]
        entity_type = entity['obj']
        entity_name = entities['text'][entity['span']['begin']:entity['span']['end']]
        
        try:
            entity_id = [id for id in entity['id'] if id.startswith("BERN")][0]
        except IndexError:
            entity_id = entity_name
        
        e.append({
            'entity_id': entity_id,
            'other_ids': other_ids,
            'entity_type': entity_type,
            'entity': entity_name
        })

    parsed_entities.append({
        'entities': e,
        'text': entities['text'],
        'text_sha256': hashlib.sha256(entities['text'].encode('utf-8')).hexdigest()
    })
    
parsed_entities

[{'text': 'The CPM!',
  'text_sha256': '07740a8ca042666ca553659e99794d2f88fbfe35966c94277de2477830442547'},
 {'text': 'is a powerful simulation method that is known for its ability to simulate a wide range of biological scenarios.',
  'text_sha256': '1b7d80918c88b7fe4e28e3f333b6d232f49c9ec61a21cab46045e3ce8282d4f7'},
 {'entities': [{'entity_id': 'single cells',
    'other_ids': ['CUI-less'],
    'entity_type': 'cell_type',
    'entity': 'single cells'}],
  'text': 'When modelling tissue environments, many researchers wish to consider both he complex morphology of single cells, and the behavior of these cells as part of larger collectives.',
  'text_sha256': 'c1a776d22bb692de4f55202d391f385dc9a9b9097389e13ce3d6fdab49e6c5d2'},
 {'text': 'The appeal of the CPM lies partly in its ability to integrate both these aspects in a single model.',
  'text_sha256': '90d7a11080f5e7bab41b6bbdd1c90ad090a86f6a9a8e6961e84943e2d4af8181'},
 {'entities': [{'entity_id': 'tumor',
    'other_ids': ['mesh:D009

In [14]:
host = 'bolt://52.91.239.25:7687'
user = 'neo4j'
password = 'house-round-forearm'
driver = GraphDatabase.driver(host,auth=(user, password))
def neo4j_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], 
columns=result.keys())

In [15]:
article_txt

"arX1v:2312.09317v1 [q-bio.TO] 14 Dec 2023\n\nA parallelized cellular Potts model that enables simulations at\ntissue scale\n\nShabaz Sultan!?, Sapna Devi?, Scott N. Mueller?, Johannes Textor!?\n\n! Data Science group, Institute for Computing and Information Sciences, Radboud University, Nijmegen,\nThe Netherlands\n\n? Department of Medical BioSciences, Radboud University Medical Center, Nijmegen, The Netherlands\n\n3 Department of Microbiology and Immunology, The University of Melbourne, at the Peter Doherty\nInstitute for Infection and Immunity, Melbourne, Victoria, Australia\n\nSummary\n\nThe Cellular Potts Model (CPM) is a widely used simulation paradigm for systems of interacting cells that\nhas been used to study scenarios ranging from plant development to morphogenesis, tumor growth and\ncell migration. Despite their wide use, CPM simulations are considered too computationally intensive\nfor three-dimensional (3D) models at organ scale. CPMs have been difficult to parallelise be

In [26]:
authors = ["Shabaz Sultan", "Sapna Devi", "Scott N. Mueller", "Johannes Textor"]
title = "A parallelized cellular Potts model that enables simulations at tissue scale"
neo4j_query("""
MERGE (b:Article {title: $title})
WITH b
UNWIND $authors AS authorName
MERGE (a:Author {name: authorName})
MERGE (a)-[:WROTE]->(b)
""", {'title': title, 'authors': authors})

""


In [27]:
neo4j_query("""
MATCH (a:Article {title: $title})
WITH a
OPTIONAL MATCH (a)-[:HAS_SENTENCE]->(s:Sentence)
WHERE s IS NULL
WITH a
UNWIND $data as row
MERGE (s:Sentence {id: row.text_sha256})
SET s.text = row.text
MERGE (a)-[:HAS_SENTENCE]->(s)
WITH s, row.entities as entities
UNWIND entities as entity
MERGE (e:Entity {id: entity.entity_id})
ON CREATE SET e.other_ids = entity.other_ids,
              e.name = entity.entity,
              e.type = entity.entity_type
MERGE (s)-[m:MENTIONS]->(e)
ON CREATE SET m.count = 1
ON MATCH SET m.count = m.count + 1
""", {'data': parsed_entities, 'title': title})

""


In [19]:
neo4j_query("""
MATCH (e:Entity)<-[:MENTIONS]-(s:Sentence)
WHERE e.name = "cells"
RETURN s.text as result
""")

,result
0,Such divergence could lead to changed simulati...
1,"The most important method, that is the main fo..."
2,The mesenchymal stem cell-based therapeutic me...


In [ ]:
model = RelTaggerModel.from_pretrained("fractalego/fewrel-zero-shot")
print("model loaded")
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
print("token loaded")
relations = ['associated', 'interacts']
extractor = RelationExtractor(model, tokenizer, relations)